# QCoDeS Example with Agilent 34400A

This notebook shows how to work with Agilent 34400A instrument with help of it's qcodes driver and conduct measurements.

In [1]:
%matplotlib nbagg
import matplotlib.pyplot as plt
import os
import numpy as np

import qcodes as qc
from qcodes.utils.validators import Enum, Strings
import qcodes.instrument_drivers.tektronix.Keithley_2600_channels as keith
import qcodes.instrument_drivers.agilent.Agilent_34400A as agi
from qcodes.dataset import initialise_or_create_database_at, load_or_create_experiment, Measurement
from qcodes.dataset.plotting import plot_dataset, plot_by_id
#from qcodes.loops import Loop

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\v-singak\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\v-singak\.qcodes\logs\210205-13016-qcodes.log


## Initialize instruments

In [ ]:
import qcodes.instrument.sims as sims
path_to_yaml = sims.__file__.replace('__init__.py', 'Keithley_2600.yaml')
k1 = keith.Keithley_2600('Keithley1', address='GPIB::1::INSTR', visalib=path_to_yaml + '@sim')
#k1 = keith.Keithley_2600('Keithley1', 'GPIB0::15::INSTR')

a1 = agi.Agilent_34400A('Agilent1', 'GPIB0::11::INSTR', setter_instr=k1)
a2 = agi.Agilent_34400A('Agilent2', 'GPIB0::6::INSTR', setter_instr=k1)

# set integration time (number of line cycles)
a1.NPLC.set(10)
a2.NPLC.set(10)

## Create the station with initialized instruments

In [ ]:
station = qc.Station()
station.add_component(k1)
station.add_component(a1)
station.add_component(a2)

## Initialize database and begin experiment

In [ ]:
tutorial_db_path = os.path.join(os.getcwd(), 'tutorial.db')
initialise_or_create_database_at(tutorial_db_path)
load_or_create_experiment(experiment_name='tutorial_exp', sample_name="no sample")

## Measurements

In [ ]:
meas1 = Measurement(station=station)
meas1.register_parameter(k1.a.volt)
meas1.register_parameter(a1.volt)

meas1.write_period = 2

with meas1.run() as datasaver:
    for set_v in np.linspace(-5, 5, 1):
        k1.a.volt.set(set_v)
        get_v = a1.volt.get()
        datasaver.add_result((k1.a.volt, set_v), (a1.volt, get_v))
dataset = datasaver.dataset
    #data = Loop(k1.a.volt[-5:5:1], 0).each(a1.volt).run(location='testsweep', overwrite=True,background=False)

_ = plot_dataset(dataset)

In [ ]:
meas2 = Measurement(station=station)
meas2.register_parameter(k1.a.volt)
meas2.register_parameter(a1.volt)
meas2.register_parameter(a2.volt)

meas2.write_period = 2

with meas2.run() as datasaver:
    for set_v in np.linspace(-5, 5, 1):
        k1.a.volt.set(set_v)
        get_v1 = a1.volt.get()
        get_v2 = a2.volt.get()
        datasaver.add_result((k1.a.volt, set_v), (a1.volt, get_v1), (a2.volt, get_v2))
run_id = datasaver.run_id
    #data = Loop(k1.a.volt[-5:5:1], 0).each(a1.volt, a2.volt).run(location='testsweep', overwrite=True,background=False)
    
_ = plot_by_id(run_id)